In [1]:
from ray.tune.registry import register_env
from custom_envs.diff_driven.gym_env.centered_gym_env.env import DiffDriveEnv

def env_creator(config):
    return DiffDriveEnv()

register_env("diffdrive_env", env_creator)


2025-05-21 11:44:02,162	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-05-21 11:44:04,330	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from ray.rllib.algorithms.maddpg import MADDPGConfig
from custom_envs.diff_driven.gym_env.centered_gym_env.config import NUM_AGENTS
from ray import rllib

env = DiffDriveEnv()
obs_space = env.observation_space
act_space = env.action_space

config = (
    MADDPGConfig()
    .environment("diffdrive_env", env_config={})
    .rollouts(num_rollout_workers=0)
    .multi_agent(
        policies={f"agent_{i}": (None, obs_space, act_space, {}) for i in range(NUM_AGENTS)},
        policy_mapping_fn=lambda agent_id, *args: agent_id,
        policies_to_train=[f"agent_{i}" for i in range(NUM_AGENTS)]
    )
    .framework("torch")
    .resources(num_gpus=1)  # ✅ Force full GPU use
    .training(
        actor_lr=1e-3,
        critic_lr=1e-3,
        train_batch_size=1024,
        tau=0.01,
        gamma=0.95,
        use_local_critic=True
    )
)


ModuleNotFoundError: No module named 'ray.rllib.algorithms.maddpg'